In [1]:
# need to read dataset from huggingface

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import time
import random
import math
import copy
import json
import re

from datasets import load_dataset


# huggingface datasets
# ! pip install datasets

c:\Users\Sayeed\anaconda3\lib\site-packages\huggingface_hub\utils\_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
def data_loader(index):
    url = 'shahidul034/text_summarization_dataset'
    url = url + str(index)
    dataset = load_dataset(url)

    path = 'text_summarization_dataset' + str(index) + '.csv'
    dataset['train'].to_csv(path)
    df = pd.read_csv(path)
    



    

In [8]:
# initialize dataframe
df = pd.DataFrame(columns=['title', 'content'])
for i in range(1, 10):
    data_loader(i)
    df
# index = 2
# url = 'shahidul034/text_summarization_dataset'
# url = url + str(index)
# dataset = load_dataset(url)



Creating CSV from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/106 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/88 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/130 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/114 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/102 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/105 [00:00<?, ?ba/s]

In [9]:
df = pd.read_csv('text_summarization_dataset2.csv')
df

,title,content
0,নিজামউদ্দিনের দরগাহ জিয়ারত করলেন প্রধানমন্ত্রী,নয়াদিল্লিতে হযরত নিজামউদ্দিন আউলিয়ার দরগাহ জিয়...
1,শাহজালালে বিমানের টয়লেটে মিলল ৪০ স্বর্ণের বার,শাহজালাল আন্তর্জাতিক বিমানবন্দরে বিমান বাংলাদে...
2,শাহজালালে বিমানের টয়লেটে মিলল ৪০ স্বর্ণের বার,"কাস্টমস কর্তৃপক্ষ সূত্র জানায়, কাস্টমস গোয়েন্..."
3,যেকোনো প্রতিষ্ঠান-কর্তৃপক্ষে নিরীক্ষা চালাতে প...,"জনসাধারণের টাকার প্রাপ্যতা, বৈধতা নিশ্চিত করতে..."
4,যেকোনো প্রতিষ্ঠান-কর্তৃপক্ষে নিরীক্ষা চালাতে প...,প্রধান বিচারপতি হাসান ফয়েজ সিদ্দিকীর লেখা ১২ ...
...,...,...
105247,"বায়তুল মোকাররমে ঈদের ৫ জামাত, প্রথমটি সকাল ৭টায়",এতে ইমামতি করবেন বায়তুল মুকাররম জাতীয় মসজিদের...
105248,"বায়তুল মোকাররমে ঈদের ৫ জামাত, প্রথমটি সকাল ৭টায়",মুকাব্বির থাকবেন মুয়াজ্জিন হাফেজ ক্বারী কাজী ...
105249,"বায়তুল মোকাররমে ঈদের ৫ জামাত, প্রথমটি সকাল ৭টায়",এই জামাতটি সকাল ১০টায় অনুষ্ঠিত হবে।পঞ্চম ও শে...
105250,ট্রেনে চেপে ঢাকায় এলো কোরবানির ৮০০ গরু,পবিত্র ঈদুল আজহা উপলক্ষে কোরবানির পশু পরিবহনের...


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 105252
    })
})

In [12]:
dataset['train'][2]

{'title': 'শাহজালালে বিমানের টয়লেটে মিলল ৪০ স্বর্ণের বার',
 'content': ' কাস্টমস কর্তৃপক্ষ সূত্র জানায়, কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের মহাপরিচালকের কাছে সংবাদ আসে সংযুক্ত আরব আমিরাতের আবুধাবি থেকে চট্টগ্রাম হয়ে আসা বিমান বাংলাদেশ এয়ারলাইনসের বিমানে অবৈধভাবে চোরাচালানকৃত স্বর্ণ বহন করা হচ্ছে। এর প্রেক্ষিতে কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের যুগ্ম মহাপরিচালক আরেফিন খানের নির্দেশে উপপরিচালক আহমেদুর রেজা চৌধুরীর নেতৃত্বে কাস্টমস গোয়েন্দার একটি দল ১০ নম্বর বোর্ডিং ব্রিজের সামনে অবস্থান নেয়।পরে কাস্টমস গোয়েন্দা দল বিমানের ভেতরে প্রবেশ করে এবং বিমানের টয়লেট থেকে বিশেষভাবে লুকিয়ে রাখা ৪০টি স্বর্ণবার উদ্ধার করে। যার মোট ওজন চর কেজি ৬৪০ গ্রাম। আটক স্বর্ণের বাজারমূল্য আনুমানিক তিন কোটি ২৪ লাখ ৮০ হাজার টাকা। এ বিষয়ে বিমানবন্দর থানায় মামলা দায়েরের প্রক্রিয়া চলছে।'}